In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pd_pro
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import gc
import lightgbm as lgbm

In [2]:
test = pd.read_csv('input/test.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32,                             
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8')  
                   

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_Id = test['ID'].values

In [4]:
#chunksize = 10 ** 7
#chunks = pd.read_csv('input/train.csv', chunksize=chunksize)
train = pd.read_csv('input/train.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32, 
                            'ConversionPayOut' : np.float32,
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8',  
                     nrows= 10000000)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len_train = train.shape[0]
y_train = train.ConversionPayOut.values
train.drop(['ConversionStatus','ConversionDate', 'ConversionPayOut'], inplace = True, axis=1)

In [6]:
data = pd.concat([train, test])

In [7]:
del train, test
gc.collect()

14

In [8]:
data.Country.fillna('other', inplace = True)
data.TrafficType.fillna('other', inplace = True)
data.Device.fillna('other', inplace = True)
data.Browser.fillna('other', inplace = True)
data.OS.fillna('other', inplace = True)
data.RefererUrl.fillna('other', inplace = True)
data.subPublisherId.fillna('other', inplace = True)
data['publisherId'] = pd.to_numeric(data['publisherId'], errors='coerce').astype(np.float32)
data['subPublisherId'] = pd.to_numeric(data['subPublisherId'], errors='coerce').astype(np.float32)

In [9]:
cols_to_encode = ['Country',  'TrafficType', 'Device','Browser', 'OS', 'RefererUrl', 'UserIp', 
                 'subPublisherId','publisherId']
le = LabelEncoder()
for col in cols_to_encode:
    data[col] = le.fit_transform(data[col]).astype(np.int32)
    

In [10]:
data['tweekday'] = data['ClickDate'].dt.weekday.astype(np.int32)
data['thour'] = data['ClickDate'].dt.hour.astype(np.int32)
data['tminute'] = data['ClickDate'].dt.minute.astype(np.int32)

In [11]:
#data.head(5)

In [12]:
ref_url_cnt = data.groupby(['UserIp'])[ 'RefererUrl'].nunique().reset_index().\
                        rename(columns = {'RefererUrl': 'ref_url_cnt'}).astype(np.int32)

In [13]:
pubId_cnt = data.groupby(['UserIp'])[ 'publisherId'].nunique().reset_index().\
                    rename(columns = {'publisherId': 'pubId_cnt'}).astype(np.int32)

In [14]:
adv_comp_cnt = data.groupby(['UserIp'])[ 'advertiserCampaignId'].nunique().reset_index().\
                             rename(columns = {'advertiserCampaignId': 'compId_cnt'}).astype(np.int32)

In [15]:
data = pd.merge(data, ref_url_cnt, how='left', on=['UserIp'])
data = pd.merge(data, pubId_cnt, how='left', on=['UserIp'])
data = pd.merge(data, adv_comp_cnt, how='left', on=['UserIp'])

In [16]:
del ref_url_cnt, pubId_cnt, adv_comp_cnt
gc.collect()

84

In [17]:
data['refUrl_user_cnt'] = data.groupby(['RefererUrl'])['ID'].transform('count').astype(np.int32)

In [18]:
data['user_cnt'] = data.groupby(['UserIp'])['ID'].transform('count').astype(np.int32)

In [19]:
data['user_week_cnt'] = data.groupby(['UserIp', 'tweekday'])['ID'].transform('count').astype(np.int32)
data['user_hr_cnt'] = data.groupby(['UserIp', 'tweekday', 'thour'])['ID'].transform('count').astype(np.int32)

In [20]:
data['user_refurl_share'] = (data['user_cnt']/ data['ref_url_cnt']).astype(np.float32)
data['user_pubid_share'] = (data['user_cnt']/ data['pubId_cnt']).astype(np.float32)
data['user_compid_share'] = (data['user_cnt']/data['compId_cnt']).astype(np.float32)

In [21]:
data.drop(['ID','ClickDate'], axis=1, inplace=True)
train = data[:len_train]
test = data[len_train:]
test.shape

(25548873, 25)

In [22]:
del data
gc.collect()

70

In [184]:
train_X, valid_X, train_y, valid_y = train_test_split(train, y_train, test_size=0.4, random_state=32)

In [145]:
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)

(8000000, 25) (8000000,) (2000000, 25) (2000000,)


In [146]:
lgb_train = lgbm.Dataset(train_X, train_y)
lgb_eval = lgbm.Dataset(valid_X, valid_y, reference=lgb_train)

In [147]:
del train_X, valid_X, train_y, valid_y

In [182]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'lambda_l1': 1000,
    'lambda_l2': 10,
    'metric': 'rmse',
    'num_leaves':106,
    'min_data_in_leaf':50,
    'max_bin':10,
    'learning_rate': 0.06,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    #'device':'gpu',
    'verbose': 0
}

In [183]:
model_lgb = lgbm.train(params,
                lgb_train,                
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=20)

[1]	valid_0's rmse: 5.59918
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 5.59916
[3]	valid_0's rmse: 5.59915
[4]	valid_0's rmse: 5.59913
[5]	valid_0's rmse: 5.59912
[6]	valid_0's rmse: 5.5991
[7]	valid_0's rmse: 5.59908
[8]	valid_0's rmse: 5.59907
[9]	valid_0's rmse: 5.59905
[10]	valid_0's rmse: 5.59904
[11]	valid_0's rmse: 5.59903
[12]	valid_0's rmse: 5.59902
[13]	valid_0's rmse: 5.59901
[14]	valid_0's rmse: 5.59901
[15]	valid_0's rmse: 5.599
[16]	valid_0's rmse: 5.59899
[17]	valid_0's rmse: 5.59898
[18]	valid_0's rmse: 5.59897
[19]	valid_0's rmse: 5.59896
[20]	valid_0's rmse: 5.59896
[21]	valid_0's rmse: 5.59895
[22]	valid_0's rmse: 5.59894
[23]	valid_0's rmse: 5.59892
[24]	valid_0's rmse: 5.59892
[25]	valid_0's rmse: 5.5989
[26]	valid_0's rmse: 5.5989
[27]	valid_0's rmse: 5.5989
[28]	valid_0's rmse: 5.59889
[29]	valid_0's rmse: 5.59889
[30]	valid_0's rmse: 5.59889
[31]	valid_0's rmse: 5.59888
[32]	valid_0's rmse: 5.59888
[33]	valid_0's rmse: 5.59

In [175]:
y_pred = model_lgb.predict(test, num_iteration=model_lgb.best_iteration)

In [176]:
sub = pd.DataFrame({'ID': test_Id, 'ConversionPayOut' : y_pred })
sub = sub[['ID','ConversionPayOut'] ]

In [177]:
sub.head(5)

,ID,ConversionPayOut
0,63367289,0.024476
1,63367290,0.049632
2,63367291,0.044863
3,63367292,0.018343
4,63367293,0.025013


In [178]:
sub['ConversionPayOut'].mean()

0.037134614476932448

In [171]:
sub.to_csv('lgbm_v15-8.csv', index=False)

In [172]:
del sub
gc.collect()

139